In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Pancreatic_Cancer/GSE236951'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene based immune profiling of vermiform appendix GALT of adult patients with pancreatic cancer or colon diseases."
!Series_summary	"Differential immune gene expression analysis in appendix GALT between patients with pancreatic cancer versus benign or malignant colon disease using the nanostring ncounter platform."
!Series_overall_design	"In this study the GALT of appendix was analyzed by RNA expression analysis of ~700 immune related genes using the Pan Cancer Immune Panel from nanostring. Non inflamed normal looking FFPE appendix tissue from sixteen pancreatic cancer patients, nine colon cancer patients, and nine benign colon disease patients was used to compare GALT immune profiles among the three groups."
Sample Characteristics Dictionary:
{0: ['disease: Pancreatic ductal adenocarcinoma', 'disease: Colon adenocarcinoma', 'disease: Benign colon disease'], 1: ['neoadjuvant therapy: None', 'neoadjuvant therapy: n.a.'], 2: ['Sex: Male', 'Sex: Fema

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Based on the background information, it is likely that this dataset contains gene expression data.
is_gene_available = True

# Determine the availability of the 'Pancreatic_Cancer' trait.
if len(set(['Pancreatic ductal adenocarcinoma', 'Colon adenocarcinoma', 'Benign colon disease'])) > 1:
    trait_row = 0

# Determine the availability of 'gender'.
if len(set(['Male', 'Female'])) > 1:
    gender_row = 2

# Determine the availability of 'age'.
if len(set(['83 years', '64 years', '59 years', '72 years', '89 years', '82 years', '75 years', '61 years', '68 years', '49 years', '71 years', '58 years', '76 years', '67 years', '52 years', '57 years', '53 years', '95 years', '55 years', '43 years', '48 years', '63 years'])) > 1:
    age_row = 3

# Define data type conversion functions.

def convert_trait(value):
    try:
        trait, val = value.split(':')
        return 1 if 'Pancreatic ductal adenocarcinoma' in val.strip() else 0
    except ValueError:
        return None

def convert_age(value):
    try:
        _, val = value.split(':')
        return int(val.strip().split()[0])  # removing "years"
    except (ValueError, IndexError):
        return None

def convert_gender(value):
    try:
        _, val = value.split(':')
        return 1 if 'Male' in val.strip() else 0
    except ValueError:
        return None

# Save metadata
save_cohort_info('GSE236951', './preprocessed/Pancreatic_Cancer/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical feature extraction if trait_row is available
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Pancreatic_Cancer', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Pancreatic_Cancer/trait_data/GSE236951.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM7587683': [1, 83, 1], 'GSM7587684': [1, 64, 1], 'GSM7587685': [1, 59, 1], 'GSM7587686': [1, 64, 1], 'GSM7587687': [1, 72, 0], 'GSM7587688': [1, 72, 1], 'GSM7587689': [1, 89, 1], 'GSM7587690': [1, 59, 0], 'GSM7587691': [1, 64, 0], 'GSM7587692': [1, 82, 0], 'GSM7587693': [1, 75, 0], 'GSM7587694': [1, 61, 1], 'GSM7587695': [1, 59, 1], 'GSM7587696': [1, 68, 1], 'GSM7587697': [1, 49, 0], 'GSM7587698': [1, 71, 1], 'GSM7587699': [0, 68, 0], 'GSM7587700': [0, 58, 0], 'GSM7587701': [0, 76, 1], 'GSM7587702': [0, 67, 1], 'GSM7587703': [0, 52, 0], 'GSM7587704': [0, 57, 1], 'GSM7587705': [0, 72, 0], 'GSM7587706': [0, 59, 0], 'GSM7587707': [0, 53, 0], 'GSM7587708': [0, 95, 0], 'GSM7587709': [0, 53, 1], 'GSM7587710': [0, 55, 1], 'GSM7587711': [0, 43, 0], 'GSM7587712': [0, 71, 0], 'GSM7587713': [0, 48, 0], 'GSM7587714': [0, 43, 1], 'GSM7587715': [0, 55, 1], 'GSM7587716': [0, 63, 0]}


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['A2M', 'ABCB1', 'ABL1', 'ADA', 'ADORA2A', 'AICDA', 'AIRE', 'AKT3',
       'ALCAM', 'AMBP', 'AMICA1', 'ANP32B', 'ANXA1', 'APOE', 'APP', 'ARG1',
       'ARG2', 'ATF1', 'ATF2', 'ATG10'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = False


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['A2M', 'ABCB1', 'ABL1', 'ADA', 'ADORA2A'], 'GB_ACC': ['NM_000014.4', 'NM_000927.3', 'NM_005157.3', 'NM_000022.2', 'NM_000675.3'], 'Analyte Type': ['mRNA', 'mRNA', 'mRNA', 'mRNA', 'mRNA'], 'CodeClass': ['Endogenous', 'Endogenous', 'Endogenous', 'Endogenous', 'Endogenous'], 'SPOT_ID': [nan, nan, nan, nan, nan]}


### Step 6: Gene Identifier Mapping

In [7]:
# Since requires_gene_mapping is False, SKIP STEP6


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Pancreatic_Cancer/gene_data/GSE236951.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data are severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Pancreatic_Cancer')

# 4. Save the cohort information.
json_path = './preprocessed/Pancreatic_Cancer/cohort_info.json'
cohort = 'GSE236951'
save_cohort_info(cohort, json_path, True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Pancreatic_Cancer/GSE236951.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Pancreatic_Cancer', the least common label is '1.0' with 16 occurrences. This represents 47.06% of the dataset.
The distribution of the feature 'Pancreatic_Cancer' in this dataset is fine.

Quartiles for 'Age':
  25%: 55.5
  50% (Median): 63.5
  75%: 71.75
Min: 43.0
Max: 95.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 17 occurrences. This represents 50.00% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

